## Max Flow Problem Example

In [1]:
using JuMP, Clp

# incidence matrix
# rows are nodes, columns are arcs, entries represent whether arc enters(-1) exits(1)

# 해당 row의 node가 -> 앞쪽에 있으면 -1, 뒤쪽에 있으면 1
#   01 02 12 13 21 24 32 34 35 43 45 
A = [1  1  0  0  0  0  0  0  0  0  0     # 0
    -1  0  1  1 -1  0  0  0  0  0  0     # 1
     0 -1 -1  0  1  1 -1  0  0  0  0     # 2
     0  0  0  -1 0  0  1  1  1 -1  0     # 3
     0  0  0  0  0 -1  0 -1  0  1  1     # 4
     0  0  0  0  0  0  0  0 -1  0 -1]    # 5

# add a dummy arc from sink (5) to source (0)
A = [A [-1;0;0;0;0;1]]

# supply and demand are all 0
b = [0, 0, 0, 0, 0, 0]

# costs should be 0 on every arc except dummy
# -1 on dummy arc
c = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1]

# capacities on each arc. make dummy arc capacity "big enough"
cap = [10, 8, 5, 5, 4, 10, 7, 6, 3, 10, 14, 100]

m = Model(Clp.Optimizer)

# variables representing how much flow we send on each arc
@variable(m, x[1:12] >= 0)

# constraints balance flow into and out of each node
@constraint(m, A*x .== b)

# don't exceed arc capacity
@constraint(m, x .<= cap)

# minimize the total cost (same as maximizing flow through dummy arc)
@objective(m, Min, sum(c[i]*x[i] for i in 1:12))

# solve this instance of the max flow problem
optimize!(m)

# print out the flow on each arc along with total flow
println("Max flow: ", -objective_value(m)) # need to take negative
println("Flow on each arc: ")
println("0-1: ", value(x[1]))
println("0-2: ", value(x[2]))
println("1-2: ", value(x[3]))
println("1-3: ", value(x[4]))
println("2-1: ", value(x[5]))
println("2-4: ", value(x[6]))
println("3-2: ", value(x[7]))
println("3-4: ", value(x[8]))
println("3-5: ", value(x[9]))
println("4-3: ", value(x[10]))
println("4-5: ", value(x[11]))
println("Dummy: ", value(x[12]))

Max flow: 15.0
Flow on each arc: 
0-1: 7.0
0-2: 8.0
1-2: 2.0
1-3: 5.0
2-1: 0.0
2-4: 10.0
3-2: 0.0
3-4: 2.0
3-5: 3.0
4-3: 0.0
4-5: 12.0
Dummy: 15.0
Coin0506I Presolve 5 (-13) rows, 11 (-1) columns and 22 (-14) elements
Clp0006I 0  Obj 0 Dual inf 1.999998 (2)
Clp0006I 7  Obj -15
Clp0000I Optimal - objective value -15
Coin0511I After Postsolve, objective -15, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -15 - 7 iterations time 0.022, Presolve 0.01


In [2]:
A

6×12 Matrix{Int64}:
  1   1   0   0   0   0   0   0   0   0   0  -1
 -1   0   1   1  -1   0   0   0   0   0   0   0
  0  -1  -1   0   1   1  -1   0   0   0   0   0
  0   0   0  -1   0   0   1   1   1  -1   0   0
  0   0   0   0   0  -1   0  -1   0   1   1   0
  0   0   0   0   0   0   0   0  -1   0  -1   1